# Graph-Aware Remasking vs WINO Baseline - GSM8K Benchmark

이 노트북은 **Graph-Aware Historical Remasking** 디코더와 **WINO** baseline을 GSM8K 벤치마크에서 비교합니다.

## 실험 설정 (논문 기준)
- **8-shot Chain-of-Thought prompting**
- **Generation Length**: 256
- **Block Length**: 128 (WINO 논문 설정)
- **평가 지표**: Accuracy, Forward passes, Remask count

In [ ]:
import os
import sys
import torch
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from transformers import AutoTokenizer

# Add current directory to path
current_dir = os.getcwd()
if current_dir not in sys.path:
    sys.path.append(current_dir)

# Import local modules
from modeling_llada import LLaDAModelLM
from configuration_llada import LLaDAConfig
from decoding import decoding_wino, decoding_graph_remask
import benchmark_utils

print("Modules loaded successfully.")

## 1. 8-shot CoT Examples

GSM8K 표준 8-shot Chain-of-Thought 예제

In [ ]:
# 8-shot CoT examples for GSM8K
FEW_SHOT_EXAMPLES = """
Q: There are 15 trees in the grove. Grove workers will plant trees in the grove today. After they are done, there will be 21 trees. How many trees did the grove workers plant today?
A: Let's think step by step. There are 15 trees originally. Then there were 21 trees after some more were planted. So there must have been 21 - 15 = 6. The answer is 6.

Q: If there are 3 cars in the parking lot and 2 more cars arrive, how many cars are in the parking lot?
A: Let's think step by step. There are originally 3 cars. 2 more cars arrive. 3 + 2 = 5. The answer is 5.

Q: Leah had 32 chocolates and her sister had 42. If they ate 35, how many pieces do they have left in total?
A: Let's think step by step. Originally, Leah had 32 chocolates. Her sister had 42. So in total they had 32 + 42 = 74. After eating 35, they had 74 - 35 = 39. The answer is 39.

Q: Jason had 20 lollipops. He gave Denny some lollipops. Now Jason has 12 lollipops. How many lollipops did Jason give to Denny?
A: Let's think step by step. Jason started with 20 lollipops. Then he had 12 after giving some to Denny. So he gave Denny 20 - 12 = 8. The answer is 8.

Q: Shawn has five toys. For Christmas, he got two toys each from his mom and dad. How many toys does he have now?
A: Let's think step by step. Shawn started with 5 toys. If he got 2 toys each from his mom and dad, then that is 4 more toys. 5 + 4 = 9. The answer is 9.

Q: There were nine computers in the server room. Five more computers were installed each day, from monday to thursday. How many computers are now in the server room?
A: Let's think step by step. There were originally 9 computers. For each of 4 days, 5 more computers were added. So 5 * 4 = 20 computers were added. 9 + 20 is 29. The answer is 29.

Q: Michael had 58 golf balls. On tuesday, he lost 23 golf balls. On wednesday, he lost 2 more. How many golf balls did he have at the end of wednesday?
A: Let's think step by step. Michael started with 58 golf balls. After losing 23 on tuesday, he had 58 - 23 = 35. After losing 2 more, he had 35 - 2 = 33 golf balls. The answer is 33.

Q: Olivia has $23. She bought five bagels for $3 each. How much money does she have left?
A: Let's think step by step. Olivia had 23 dollars. 5 bagels for 3 dollars each will be 5 x 3 = 15 dollars. So she has 23 - 15 dollars left. 23 - 15 is 8. The answer is 8.
""".strip()

def create_cot_prompt(question):
    """Create 8-shot CoT prompt for GSM8K"""
    return f"{FEW_SHOT_EXAMPLES}\n\nQ: {question}\nA: Let's think step by step."

print("8-shot CoT template created.")
print(f"\nExample prompt length: ~{len(FEW_SHOT_EXAMPLES.split())} words")

## 2. Load Model

In [ ]:
LOCAL_MODEL_PATH = "../Grok-1-LLaDA-8B"
HF_MODEL_ID = "GSAI-ML/LLaDA-8B-Base"

model_path = HF_MODEL_ID
if os.path.exists(LOCAL_MODEL_PATH):
    model_path = LOCAL_MODEL_PATH
    print(f"Using local model: {model_path}")
else:
    print(f"Using HuggingFace model: {model_path}")

config = LLaDAConfig.from_pretrained(model_path)
model = LLaDAModelLM.from_pretrained(model_path, config=config, torch_dtype="auto")

if torch.cuda.is_available():
    model.cuda()
model.eval()

tokenizer = AutoTokenizer.from_pretrained(model_path)
print("Model loaded successfully.")

## 3. Load GSM8K Dataset

In [ ]:
# Configuration - 논문 설정에 맞춤
N_SAMPLES = 50  # Number of GSM8K samples to test
GEN_LENGTH = 256  # WINO 논문 설정
BLOCK_LENGTH = 128  # WINO 논문 설정 (256에서 128로 변경)

print(f"Loading GSM8K dataset (N={N_SAMPLES})...")
gsm8k_data = benchmark_utils.load_gsm8k(n_samples=N_SAMPLES)
print(f"Loaded {len(gsm8k_data)} samples.")

# Show example with CoT prompt
if gsm8k_data:
    example_question = gsm8k_data[0]['question']
    example_prompt = create_cot_prompt(example_question)
    print("\n=== Example CoT Prompt ===")
    print(example_prompt[:500] + "...")
    print(f"\nGround truth: {gsm8k_data[0]['ground_truth']}")

## 4. Run Benchmark Comparison

In [ ]:
results = []
mask_id = 126336

for idx, item in enumerate(gsm8k_data):
    question = item['question']
    ground_truth = item['ground_truth']
    
    # Create 8-shot CoT prompt
    prompt_text = create_cot_prompt(question)
    
    # Encode prompt
    prompt_tokens = tokenizer.encode(prompt_text, return_tensors='pt').to(model.device)
    
    print(f"\n[{idx+1}/{len(gsm8k_data)}] Processing...")
    print(f"  Prompt length: {prompt_tokens.shape[1]} tokens")
    
    # === WINO Baseline ===
    try:
        output_wino, steps_wino = decoding_wino(
            model=model,
            prompt=prompt_tokens,
            gen_length=GEN_LENGTH,
            block_length=BLOCK_LENGTH,
            temperature=0.0,
            mask_id=mask_id,
            threshold=0.6,
            threshold_back=0.9
        )
        text_wino = tokenizer.decode(output_wino[0, prompt_tokens.shape[1]:], skip_special_tokens=True)
        correct_wino = benchmark_utils.check_correctness(text_wino, ground_truth, "Math")
        
        print(f"  WINO: {steps_wino} steps, Correct: {correct_wino}")
        if not correct_wino:
            extracted = benchmark_utils.extract_number(text_wino)
            print(f"    Extracted: {extracted}, Expected: {ground_truth}")
    except Exception as e:
        print(f"  WINO failed: {e}")
        steps_wino = -1
        correct_wino = False
        text_wino = ""
    
    # === Graph-Aware Remasking ===
    try:
        output_graph, stats_graph = decoding_graph_remask(
            model=model,
            prompt=prompt_tokens,
            gen_length=GEN_LENGTH,
            block_length=BLOCK_LENGTH,
            temperature=0.0,
            mask_id=mask_id,
            threshold_forward=0.6,
            threshold_back=0.9,
            resp_threshold=0.3,
            gamma_decay=0.95,
            use_attention_layers=[-1],
            top_k_attention=10,
            max_remask_ratio=0.3
        )
        text_graph = tokenizer.decode(output_graph[0, prompt_tokens.shape[1]:], skip_special_tokens=True)
        correct_graph = benchmark_utils.check_correctness(text_graph, ground_truth, "Math")
        steps_graph = stats_graph['total_steps']
        remasks_graph = stats_graph['total_remasks']
        
        print(f"  Graph: {steps_graph} steps, {remasks_graph} remasks, Correct: {correct_graph}")
        if not correct_graph:
            extracted = benchmark_utils.extract_number(text_graph)
            print(f"    Extracted: {extracted}, Expected: {ground_truth}")
    except Exception as e:
        print(f"  Graph-Remask failed: {e}")
        steps_graph = -1
        remasks_graph = -1
        correct_graph = False
        text_graph = ""
    
    # Store results
    results.append({
        'question': question,
        'ground_truth': ground_truth,
        'wino_correct': 1 if correct_wino else 0,
        'wino_steps': steps_wino,
        'graph_correct': 1 if correct_graph else 0,
        'graph_steps': steps_graph,
        'graph_remasks': remasks_graph,
        'wino_output': text_wino,
        'graph_output': text_graph
    })

# Create DataFrame
df_results = pd.DataFrame(results)
print("\n=== Benchmark Complete ===")

## 5. Analyze Results

In [ ]:
# Summary statistics
print("\n=== Summary Statistics ===")
print(f"\nWINO Baseline:")
wino_acc = df_results['wino_correct'].mean()
print(f"  Accuracy: {wino_acc:.2%} ({df_results['wino_correct'].sum()}/{len(df_results)})")
print(f"  Avg Steps: {df_results['wino_steps'].mean():.1f}")

print(f"\nGraph-Aware Remasking:")
graph_acc = df_results['graph_correct'].mean()
print(f"  Accuracy: {graph_acc:.2%} ({df_results['graph_correct'].sum()}/{len(df_results)})")
print(f"  Avg Steps: {df_results['graph_steps'].mean():.1f}")
print(f"  Avg Remasks: {df_results['graph_remasks'].mean():.1f}")

print(f"\nImprovement:")
acc_diff = graph_acc - wino_acc
print(f"  Accuracy Δ: {acc_diff:+.2%}")
step_diff = df_results['graph_steps'].mean() - df_results['wino_steps'].mean()
print(f"  Steps Δ: {step_diff:+.1f}")

print(f"\n논문 비교:")
print(f"  WINO 논문 보고: 73.24% → 75.82% (+2.58%)")
print(f"  현재 실험: {wino_acc:.2%} → {graph_acc:.2%} ({acc_diff:+.2%})")

## 6. Visualization

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Accuracy comparison
ax1 = axes[0]
accuracies = [wino_acc, graph_acc]
bars = ax1.bar(['WINO', 'Graph-Remask'], accuracies, color=['#3498db', '#e74c3c'])
ax1.set_ylabel('Accuracy')
ax1.set_title('GSM8K Accuracy Comparison (8-shot CoT)')
ax1.set_ylim([0, 1])
ax1.axhline(0.7582, color='green', linestyle='--', linewidth=2, label='WINO Paper (75.82%)')
for i, v in enumerate(accuracies):
    ax1.text(i, v + 0.02, f"{v:.2%}", ha='center', fontweight='bold')
ax1.legend()

# Steps comparison
ax2 = axes[1]
steps = [df_results['wino_steps'].mean(), df_results['graph_steps'].mean()]
ax2.bar(['WINO', 'Graph-Remask'], steps, color=['#3498db', '#e74c3c'])
ax2.set_ylabel('Average Steps')
ax2.set_title('Decoding Steps Comparison')
for i, v in enumerate(steps):
    ax2.text(i, v + 1, f"{v:.1f}", ha='center', fontweight='bold')

# Remask count
ax3 = axes[2]
ax3.hist(df_results['graph_remasks'], bins=20, color='#e74c3c', alpha=0.7, edgecolor='black')
ax3.set_xlabel('Remask Count')
ax3.set_ylabel('Frequency')
ax3.set_title('Graph-Remask: Remask Distribution')
ax3.axvline(df_results['graph_remasks'].mean(), color='red', linestyle='--', linewidth=2, 
           label=f"Mean: {df_results['graph_remasks'].mean():.1f}")
ax3.legend()

plt.tight_layout()
plt.show()

## 7. Failure Case Analysis

In [ ]:
# Find cases where WINO failed but Graph succeeded
graph_wins = df_results[(df_results['wino_correct'] == 0) & (df_results['graph_correct'] == 1)]
print(f"\n=== Graph-Remask Wins: {len(graph_wins)} cases ===")
if len(graph_wins) > 0:
    for idx, row in graph_wins.head(3).iterrows():
        print(f"\n--- Case {idx+1} ---")
        print(f"Question: {row['question'][:100]}...")
        print(f"Ground Truth: {row['ground_truth']}")
        print(f"WINO: {benchmark_utils.extract_number(row['wino_output'])}")
        print(f"Graph: {benchmark_utils.extract_number(row['graph_output'])}")

# Find cases where WINO succeeded but Graph failed
wino_wins = df_results[(df_results['wino_correct'] == 1) & (df_results['graph_correct'] == 0)]
print(f"\n=== WINO Wins: {len(wino_wins)} cases ===")
if len(wino_wins) > 0:
    for idx, row in wino_wins.head(3).iterrows():
        print(f"\n--- Case {idx+1} ---")
        print(f"Question: {row['question'][:100]}...")
        print(f"Ground Truth: {row['ground_truth']}")
        print(f"WINO: {benchmark_utils.extract_number(row['wino_output'])}")
        print(f"Graph: {benchmark_utils.extract_number(row['graph_output'])}")

## 8. Save Results

In [ ]:
# Save to CSV
output_file = "graph_remask_vs_wino_gsm8k_8shot_results.csv"
df_results.to_csv(output_file, index=False)
print(f"\nResults saved to {output_file}")

# Save summary
summary = {
    'config': {
        'n_samples': N_SAMPLES,
        'gen_length': GEN_LENGTH,
        'block_length': BLOCK_LENGTH,
        'few_shot': '8-shot CoT'
    },
    'wino': {
        'accuracy': wino_acc,
        'avg_steps': df_results['wino_steps'].mean()
    },
    'graph': {
        'accuracy': graph_acc,
        'avg_steps': df_results['graph_steps'].mean(),
        'avg_remasks': df_results['graph_remasks'].mean()
    },
    'improvement': {
        'accuracy_delta': acc_diff,
        'steps_delta': step_diff
    }
}

import json
with open('benchmark_summary.json', 'w') as f:
    json.dump(summary, f, indent=2)
print("Summary saved to benchmark_summary.json")